# Lab 03 - Spark SQL

### Solving with RDD data structure
#### 1. Input Files

#### 1.1 register.csv

In [1]:
inputpath01 = "/data/students/bigdata_internet/lab3/register.csv"

In [2]:
# Read the content of the input file
register = sc.textFile(inputpath01)

In [3]:
# Parse the data
register_data = register.map(lambda line: line.split('\t'))

In [ ]:
register_header = register_data.first()
new_rdd = register_data.filter(lambda x: x!= register_header)

#### Questions:
##### 1.1.1 How many rows of data we obtain before and after the data cleaning above?

In [5]:
# Count the number of rows before data cleaning
num_rows_before = new_rdd.count()
print ("The total rows before cleaning data is:", num_rows_before) 

[Stage 1:=======================================>                   (4 + 2) / 6]

The total rows before cleaning data is: 25319028


In [6]:
# Clean rows where used_slots = 0 or free_slots = 0
cleanRowsRDD = new_rdd.filter((lambda record: record[2] != "0" and record[3] != "0"))

claen_data = cleanRowsRDD.count()
print ("The total rows after cleaning data is:", claen_data) 

[Stage 2:=======================================>                   (4 + 2) / 6]

The total rows after cleaning data is: 19337258


In [7]:
# Task 2: Get rows where used_slots = 0 or free_slots = 0
RowsRDD = new_rdd.filter(lambda record: record[2] == "0" and record[3] == "0")

with0 = RowsRDD.count()
print ("There are", claen_data, "rows with value of '0'") 

[Stage 3:=================================================>         (5 + 1) / 6]

There are 19337258 rows with value of '0'


#### 1.2 stations.csv

In [5]:
inputpath02 = "/data/students/bigdata_internet/lab3/stations.csv"

In [6]:
# Read the content of the input file
stations = sc.textFile(inputpath02)

In [7]:
# Parse the data
stations_data = stations.map(lambda line: line.split('\t'))

In [ ]:
header = stations_data.first()
stationsRDD= stations_data.filter(lambda x: x!= header)

### 2. Exercise

#### Questions:
##### 2.1 Computes the criticality value C(Si, Tj) for each pair C(Si, Tj)
##### 2.2 Selects only the critical pairs C(Si, Tj) having a criticality value C(Si, Tj) greater than a minimum threshold. The minimum criticality threshold is a float between 0 and 1 passed as an argument of the application.
##### 2.3. Order the results by increasing criticality. If there are two or more records characterized by the same criticality value, consider the station id value (in ascending order). If also the station is the same, consider the day of the week (ascending from Monday to Sunday) and finally the hour (ascending from 0 to 23)
##### 2.4. Store the sorted critical pairs  C(Si, Tj) in the output folder (also an argument of the application), by using a csv files (with header), where columns are separated by "tab". 
##### 2.5 How many critical pairs do you obtain? Report also the complete output result of the applications.

In [9]:
# extracting weekday and hour from the timestamp

from datetime import datetime
dayHourRDD = new_rdd.map(lambda x: (x[0],(datetime.strftime(datetime.strptime(x[1], "%Y-%m-%d %H:%M:%S"),"%A %H").split(" ")[0],
                                          datetime.strftime(datetime.strptime(x[1], "%Y-%m-%d %H:%M:%S"),"%A %H").split(" ")[1]),x[2],x[3]))

In [10]:
groupedRDD = dayHourRDD.groupBy(lambda x: (x[0],x[1]))

In [11]:
# converting to a list
resultRDD = groupedRDD.map(lambda x: (x[0], list(x[1])))

In [12]:
# conuting the number of "0" slots in each group to calculate the criticality
def zero_counter(items):
    total = 0;
    for i in items:
        if i[3] == '0':
            total+=1
    return total

In [13]:
newRDD = resultRDD.map(lambda x: (x[0],x[1],len(x[1])))

In [14]:
# call the function to get the zeros of each group
zeroSlots = newRDD.map(lambda x: ( x[0], x[1], x[2], zero_counter(x[1]) ))

In [15]:
# calculating the criticality
criticality = zeroSlots.map(lambda x: ( x[0], x[1], x[2], x[3], x[3]/x[2]))

In [16]:
##import sys
# Define the minimum criticality threshold (replace 0.5 with your desired threshold)
min_threshold = 0.6

In [17]:
thresholdRDD = criticality.filter(lambda x: x[4]>min_threshold)

In [18]:
weekdays = { 'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5,'Saturday': 6, 'Sunday': 7}

In [19]:
sortedRDD = thresholdRDD.sortBy(lambda x: (x[4], x[0][0], weekdays.get(x[0][1][0]),x[0][1][1]))

In [20]:
# ('stationid', ('weekday', 'hour', criticality))
organizedRDD = sortedRDD.map(lambda x: (x[0][0],(x[1][0][1][0],x[1][0][1][1], x[4])))

In [29]:
organizedRDD.take(3)

[('221', ('Friday', '10', 0.6010016694490818)),
 ('221', ('Friday', '09', 0.6016666666666667)),
 ('221', ('Friday', '20', 0.6016666666666667))]

In [21]:
pairRDD = stationsRDD.map(lambda x : (x[0],(x[1],x[2])))

In [22]:
pairRDD.take(3)

[('1', ('2.180019', '41.397978')),
 ('2', ('2.176414', '41.394381')),
 ('3', ('2.181164', '41.393750'))]

In [23]:
# ('stationid', (('weekday', 'hour', criticality), ('long', 'lot')))
joinedRDD = organizedRDD.join(pairRDD)

In [24]:
#and each row looks like this ('10', '2.185206', '41.384875', 'Saturday', '00', 0.622107969151671)
resultRDD = joinedRDD.map(lambda x: (x[0],x[1][1][0],x[1][1][1],x[1][0][0],x[1][0][1], x[1][0][2]))


In [25]:
#final result
resultRDD.take(3)

[('10', '2.185206', '41.384875', 'Saturday', '00', 0.6230769230769231),
 ('221', '2.160747', '41.401806', 'Friday', '10', 0.6010016694490818),
 ('221', '2.160747', '41.401806', 'Friday', '09', 0.6016666666666667)]

In [26]:
# adding the header
header = ("Station", "Lattitude", "Longtitude", "Weekday", "Hour", "Criticality")
headerRDD = sc.parallelize([header])

In [29]:
#we make a union to mix two rdds
ResultRDD= headerRDD.union(resultRDD)

In [30]:
saveRDD = ResultRDD.map(lambda x: str(x[0])+'\t'+str(x[1])+'\t'+str(x[2])+'\t'+str(x[3])+'\t'+str(x[4])+'\t'+str(x[5]))
#saving  the final result as csv file
saveRDD.coalesce(1).saveAsTextFile("lab3_s302866_RDDresult.csv/")

#     ********************************************************************
#     ********************************************************************
#     ********************************************************************

# Solving with SQL
#### 1. Input Files

#### 1.1 register.csv

In [24]:
inputpath01 = "/data/students/bigdata_internet/lab3/register.csv"

In [25]:
# Read the content of the input file and store it into a DataFrame
dfReadings=spark.read.option("sep", "\t").load(inputpath01, format = "csv", header = True, inferSchema = True)

In [26]:
# Assign the “table name” readings to dfReadings
dfReadings.createOrReplaceTempView("registerSQL")

#### Questions:
##### 1.1.1 How many rows of data we obtain before and after the data cleaning above?

In [27]:
# Count the number of rows before data cleaning
num_rows_before = dfReadings.count()
print ("The total rows before cleaning data is:", num_rows_before) 

[Stage 14:================================================>         (5 + 1) / 6]

The total rows before cleaning data is: 25319028


In [28]:
# Clean rows where used_slots = 0 or free_slots = 0
cleanRows=spark.sql("SELECT used_slots, free_slots FROM registerSQL WHERE used_slots != 0 AND free_slots != 0")

In [8]:
cleanRows.count()

19337258

#### 1.2 stations.csv

In [29]:
inputpath02 = "/data/students/bigdata_internet/lab3/stations.csv"

In [41]:
# Read the content of the input file
dfStation = spark.read.option("sep", "\t").load(inputpath02, format = "csv", header = True, inferSchema = True)

In [42]:
dfStation.createOrReplaceTempView("station")

### 2. Exercise

#### Questions:
##### 2.1 Computes the criticality value C(Si, Tj) for each pair C(Si, Tj)
##### 2.2 Selects only the critical pairs C(Si, Tj) having a criticality value C(Si, Tj) greater than a minimum threshold. The minimum criticality threshold is a float between 0 and 1 passed as an argument of the application.
##### 2.3. Order the results by increasing criticality. If there are two or more records characterized by the same criticality value, consider the station id value (in ascending order). If also the station is the same, consider the day of the week (ascending from Monday to Sunday) and finally the hour (ascending from 0 to 23)
##### 2.4. Store the sorted critical pairs  C(Si, Tj) in the output folder (also an argument of the application), by using a csv files (with header), where columns are separated by "tab". 
##### 2.5 How many critical pairs do you obtain? Report also the complete output result of the applications.

In [43]:
from pyspark.sql.functions import hour, date_format, col

# Extract day of the week and hour from the timestamp
dfReadings = dfReadings.withColumn("day_of_week", date_format("timestamp", "EEEE"))
dfReadings = dfReadings.withColumn("hour", hour("timestamp"))

In [33]:
dfReadings.show()

[Stage 15:>                                                         (0 + 1) / 1]

+-------+-------------------+----------+----------+-----------+----+
|station|          timestamp|used_slots|free_slots|day_of_week|hour|
+-------+-------------------+----------+----------+-----------+----+
|      1|2008-05-15 10:01:00|         0|        18|   Thursday|  10|
|      1|2008-05-15 10:02:00|         0|        18|   Thursday|  10|
|      1|2008-05-15 10:04:00|         0|        18|   Thursday|  10|
|      1|2008-05-15 10:06:00|         0|        18|   Thursday|  10|
|      1|2008-05-15 10:08:00|         0|        18|   Thursday|  10|
|      1|2008-05-15 10:10:00|         0|        18|   Thursday|  10|
|      1|2008-05-15 10:12:00|         0|        18|   Thursday|  10|
|      1|2008-05-15 10:14:00|         0|        18|   Thursday|  10|
|      1|2008-05-15 10:16:00|         0|        18|   Thursday|  10|
|      1|2008-05-15 10:18:00|         0|        18|   Thursday|  10|
|      1|2008-05-15 10:20:00|         2|        16|   Thursday|  10|
|      1|2008-05-15 10:22:00|     

In [39]:
dfReadings.printSchema()

root
 |-- station: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- used_slots: integer (nullable = true)
 |-- free_slots: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- hour: integer (nullable = true)



In [44]:
from pyspark.sql import functions as func

zeroSlotsDF = dfReadings.withColumn("zeros", func.when(cleanRows["free_slots"] == 0, 1).otherwise(0))
zeroSlotsDF.createOrReplaceTempView("zeroRegister")

In [45]:
criticality = zeroSlotsDF.groupBy(["station", "day_of_week", "hour"]) \
       .agg(func.sum("zeros").alias("sum_zeros"), func.count("free_slots").alias("count_free_slots")) \
       .selectExpr("station", "day_of_week", "hour", "sum_zeros / count_free_slots as criticality")

criticality.show()


[Stage 20:======================================>                   (4 + 2) / 6]

+-------+-----------+----+--------------------+
|station|day_of_week|hour|         criticality|
+-------+-----------+----+--------------------+
|    263|  Wednesday|  23|  0.1634349030470914|
|    264|   Thursday|  20|0.003333333333333...|
|    265|     Friday|  13|                 0.0|
|    266|     Friday|  10|0.001666666666666...|
|    266|  Wednesday|  10|0.001754385964912...|
|    268|   Thursday|  21| 0.05175292153589316|
|    269|     Sunday|   5|                 0.0|
|    270|   Saturday|  15|                 0.0|
|    270|     Sunday|  11|                 0.0|
|    270|   Thursday|   8|                 0.0|
|    272|   Saturday|  11|0.001760563380281...|
|    273|     Sunday|  15|0.001754385964912...|
|    273|    Tuesday|  13|0.001675041876046...|
|    277|     Sunday|   9| 0.10526315789473684|
|    277|  Wednesday|  12|0.001814882032667...|
|    277|     Monday|   1| 0.08333333333333333|
|    278|     Sunday|  22| 0.22777777777777777|
|    279|   Saturday|   5|0.001757469244

In [46]:
#minimum criticality is 0.6
MinimumCriticality = 0.6
# now we filter the groups which have criticality over the min threshold
thresholdDF = criticality.select("*").where(criticality["criticality"] >= 0.6)

In [47]:
thresholdDF.show()

+-------+-----------+----+------------------+
|station|day_of_week|hour|       criticality|
+-------+-----------+----+------------------+
|     58|     Sunday|  23|             0.625|
|    221|     Friday|  20|0.6143572621035058|
|    221|     Sunday|  20|0.6748681898066784|
|    221|   Saturday|   7|0.6210526315789474|
|    221|     Sunday|  21|0.7095070422535211|
|    221|     Sunday|  10|0.6203866432337434|
|    221|   Thursday|  17|0.6494156928213689|
|    221|     Monday|   3|0.6256590509666081|
|    221|     Sunday|  13|0.6731107205623902|
|    221|   Saturday|   6|0.6239015817223199|
|    221|     Sunday|  15|0.6771929824561403|
|    221|   Thursday|  13|0.6541737649063032|
|    221|   Thursday|  18|0.6444073455759599|
|     10|     Friday|  22|0.6230769230769231|
|    221|     Sunday|  11|0.6214788732394366|
|    221|    Tuesday|   3|0.6263157894736842|
|    221|     Sunday|   4|0.6298245614035087|
|    221|   Saturday|   8|0.6210526315789474|
|    221|     Sunday|   8|0.625659

In [65]:
thresholdDF.printSchema()

root
 |-- station: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- criticality: double (nullable = true)



In [49]:
dfStation.printSchema()

root
 |-- id: integer (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- name: string (nullable = true)



In [50]:
dfStation.columns

['id', 'longitude', 'latitude', 'name']

In [40]:
thresholdDF.columns

['station', 'day_of_week', 'hour', 'criticality']

In [51]:
# Perform the join using the correct column names
joinedDfs = thresholdDF.join(dfStation, thresholdDF.station == dfStation.id).select(
    thresholdDF.station, dfStation.longitude, dfStation.latitude, 
    thresholdDF.day_of_week, thresholdDF.hour, thresholdDF.criticality
)


In [64]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql import *

weekdays = { 'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5,'Saturday': 6, 'Sunday': 7}
# spark.udf.register("weekday_order", lambda x: weekdays.get(x), IntegerType()) //once worked then didn't
weekday_order = udf(lambda x: weekdays.get(x), IntegerType())

In [65]:
# create a column named weekday_id and the data is generated using weekday_order UDF

joined = joined_dfs.withColumn("weekday_id", weekday_order(joined_dfs.day_of_week))

In [66]:
sortedDf = joined.sort("criticality","station","weekday_id","hour",ascending=[True,True,True,True])

In [67]:
# weekday_id column is not needed so we drop it
final = sortedDf.drop("weekday_id")

In [68]:
# to add the header we create an rdd with the column names
#obviously there are different way to do that
header = ("Station", "Lattitude", "Longtitude", "Weekday", "Hour", "Criticality")
header_rdd = sc.parallelize([header])

#first we change df to rdd
df_to_rdd = final.rdd

#we make a union to mix two rdds
result_rdd = header_rdd.union(df_to_rdd)


#cast data to string then add \t as seperator
saveable_rdd = result_rdd.map(lambda x: str(x[0])+'\t'+str(x[1])+'\t'+str(x[2])+'\t'+str(x[3])+'\t'+str(x[4])+'\t'+str(x[5]))

In [69]:
#saving  the final result as csv file
saveable_rdd.coalesce(1).saveAsTextFile("lab3_s308885_final_result.csv/")